<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=07b6cc7184c6f7a672f17ad6425ce8d29bf2d8f2a847f089fef77107c23f0888
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-15 10:19:27
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.39 C
-------------------
Today PnL: 55.62 K (0.4%)
Current PnL: -22.64 L (-15.08%)
CY Booked + Current PnL: -10.74 L (-7.16%)
-------------------
Total profit:  1.39 L
Total loss:  -24.03 L
-------------------
Total Booked + Current PnL: 16.24 L (13.19%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.57%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 89.83 L (64.7%)
Deployed:  1.23 C
Current:  1.39 C
CAGR/XIRR %: 7.14%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-20.47,54.0,X-LC,3.70,212487.0,7987.0,15299.0,-0.08,3.91,7.20,11.39,10.0,0.52,1.54,13.21,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,98.08,53.0,M-SC,1.51,85314.0,-15463.0,15553.0,0.91,-15.34,18.23,0.09,245.0,-0.99,0.62,12.08,OX40N,NTT,DURABLES
33,ICICIGI,2260.25,-14.31,65.0,X-MC,1.68,147556.0,11423.0,19699.0,7.50,8.39,13.35,22.86,91.0,0.58,1.07,22.06,X40,ATH,INSURANCE
83,VOLTAS,1530.00,-0.52,53.0,X-MC,1.03,208665.0,16923.0,20825.0,-0.04,8.83,9.98,19.69,99.0,0.81,1.51,16.22,XY25,NTT,AC
56,RELIANCE,1533.00,-14.04,49.0,X-LC,2.97,215140.0,4774.0,24010.0,0.23,2.27,11.16,13.68,37.0,0.20,1.56,19.17,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,ICICIGI,2260.25,-14.31,65.0,X-MC,1.68,147556.0,11423.0,19699.0,7.50,8.39,13.35,22.86,91.0,0.58,1.07,22.06,X40,ATH,INSURANCE
43,JIOFIN,387.00,-1.23,55.0,X-LC,12.24,268320.0,955.0,64504.0,2.09,0.36,24.04,24.48,48.0,0.01,1.95,55.53,XY24,BTT,FINANCE
16,CAMS,5211.76,-4.94,50.0,X-SC,1.18,108002.0,5998.0,37930.0,2.04,5.88,35.12,43.06,122.0,0.16,0.78,24.00,X40N,ATH,MISC
49,NESTLEIND,1377.00,-8.99,56.0,X-LC,2.29,280238.0,14812.0,41980.0,1.88,5.58,14.98,21.40,11.0,0.35,2.03,12.59,XY25,NTT,FMCG
47,LTIM,7230.20,1.16,67.0,H-LC,3.36,250650.0,4499.0,74719.0,1.85,1.83,29.81,32.18,16.0,0.06,1.82,38.53,X200,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,ICICIPRULI,790.0,-23.40,38.0,X-MC,2.13,131882.0,-3906.0,46660.0,-2.42,-2.88,35.38,31.48,107.0,-0.08,0.96,8.89,X40,ATH,INSURANCE
15,CAMPUS,393.0,-24.24,53.0,M-SC,2.67,157898.0,-9877.0,65717.0,-2.34,-5.89,41.62,33.28,210.0,-0.15,1.15,22.38,XY24,NTT,FOOTWEAR
46,LAOPALA,464.0,95.46,31.0,H-SC,1.51,89439.0,-41666.0,92909.0,-1.58,-31.78,103.88,39.09,142.0,-0.45,0.65,15.19,AR,NTT,CERAMICS
7,ATULAUTO,844.0,3629.73,50.0,M-SC,4.73,167484.0,-29648.0,119483.0,-1.55,-15.04,71.34,45.57,236.0,-0.25,1.22,18.74,XY24,NTT,AUTO
57,REPCOHOME,880.0,-58.27,57.0,H-SC,2.70,238023.0,-48482.0,305812.0,-1.26,-16.92,128.48,89.82,134.0,-0.16,1.73,25.93,XY24,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,68.0,M-SC,17.32,148338.0,-882.0,145060.0,0.03,-0.59,97.79,96.62,208.0,-0.01,1.08,60.53,XY25,NTT,FINANCE
59,SAIL,228.00,41.94,47.0,M-MC,12.27,223998.0,-964.0,168626.0,1.02,-0.43,75.28,74.53,192.0,-0.01,1.63,31.69,XY24,BTT,STEEL
36,INDIAMART,4810.62,-55.57,37.0,H-SC,8.27,123803.0,467.0,131157.0,0.05,0.38,105.94,106.72,119.0,0.00,0.90,23.84,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,105.37,57.0,H-SC,11.49,129627.0,-12852.0,32368.0,0.90,-9.02,24.97,13.70,163.0,-0.40,0.94,53.00,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,98.08,53.0,M-SC,1.51,85314.0,-15463.0,15553.0,0.91,-15.34,18.23,0.09,245.0,-0.99,0.62,12.08,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,48.0,H-SC,1.84,220545.0,-49122.0,85461.0,-0.16,-18.22,38.75,13.47,138.0,-0.57,1.60,12.14,XY24,NTT,PAINTS
17,CERA,9475.0,-23.39,42.0,H-SC,2.10,139806.0,-36097.0,78124.0,-0.20,-20.52,55.88,23.89,149.0,-0.46,1.01,20.74,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.72,43.0,H-MC,7.02,101792.0,-30043.0,72262.0,0.42,-22.79,70.99,32.03,98.0,-0.42,0.74,14.02,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,47.0,M-LC,6.32,205288.0,2118.0,40011.0,0.80,1.04,19.49,20.74,55.0,0.05,1.49,6.64,XY25,NTT,FINANCE
31,HINDZINC,730.22,36.19,68.0,M-LC,8.88,221763.0,16687.0,95890.0,0.82,8.14,43.24,54.89,52.0,0.17,1.61,34.19,X5K,ATH,METALS
85,WIPRO,420.00,-11.98,56.0,M-LC,5.77,154926.0,3981.0,105474.0,0.59,2.64,68.08,72.51,53.0,0.04,1.12,8.82,XR,NTT,IT
78,UNIONBANK,163.00,-13.18,55.0,M-LC,9.50,157838.0,16998.0,27495.0,1.33,12.07,17.42,31.59,66.0,0.62,1.15,41.75,XY24,NTT,BANKS
11,BANKINDIA,190.00,-25.47,61.0,H-MC,12.70,192988.0,13180.0,99234.0,0.95,7.33,51.42,62.52,88.0,0.13,1.40,43.92,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,36.19,68.0,M-LC,8.88,221763.0,16687.0,95890.0,0.82,8.14,43.24,54.89,52.0,0.17,1.61,34.19,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,61.0,H-MC,12.70,192988.0,13180.0,99234.0,0.95,7.33,51.42,62.52,88.0,0.13,1.40,43.92,XR,NTT,BANKS
85,WIPRO,420.00,-11.98,56.0,M-LC,5.77,154926.0,3981.0,105474.0,0.59,2.64,68.08,72.51,53.0,0.04,1.12,8.82,XR,NTT,IT
36,INDIAMART,4810.62,-55.57,37.0,H-SC,8.27,123803.0,467.0,131157.0,0.05,0.38,105.94,106.72,119.0,0.00,0.90,23.84,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-49.25,29.0,M-SC,4.06,85792.0,-5706.0,84454.0,-0.88,-6.24,98.44,86.07,223.0,-0.07,0.62,24.59,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,12.0,X-LC,3.32,142822.0,-133701.0,243769.0,-0.51,-48.35,170.68,39.81,54.0,-0.55,1.04,0.00,XY24,NTT,AUTO
51,QUESS,986.00,-48.21,30.0,X-SC,38.41,51410.0,-13596.0,164522.0,0.26,-20.91,320.02,232.18,198.0,-0.08,0.37,0.26,XY24,NTT,MISC
28,HAVELLS,2069.16,-8.16,33.0,X-MC,1.52,227666.0,-20065.0,95119.0,0.71,-8.10,41.78,30.30,92.0,-0.21,1.65,4.12,X40,ATH,ELECTRICAL
12,BATAINDIA,2096.00,-37.72,34.0,X-SC,4.42,93168.0,-35502.0,80805.0,-0.15,-27.59,86.73,35.20,219.0,-0.44,0.68,6.94,X40,NTT,FOOTWEAR
55,RELAXO,1176.00,-41.43,34.0,X-SC,3.76,79815.0,-65345.0,141273.0,-0.57,-45.02,177.00,52.31,136.0,-0.46,0.58,6.84,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.25,44.0,H-LC,0.91,155500.0,-30595.0,78077.0,0.09,-16.44,50.21,25.51,15.0,-0.39,1.13,14.59,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,-0.52,53.0,X-MC,1.03,208665.0,16923.0,20825.0,-0.04,8.83,9.98,19.69,99.0,0.81,1.51,16.22,XY25,NTT,AC
28,HAVELLS,2069.16,-8.16,33.0,X-MC,1.52,227666.0,-20065.0,95119.0,0.71,-8.10,41.78,30.30,92.0,-0.21,1.65,4.12,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-14.31,65.0,X-MC,1.68,147556.0,11423.0,19699.0,7.50,8.39,13.35,22.86,91.0,0.58,1.07,22.06,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-8.99,56.0,X-LC,2.29,280238.0,14812.0,41980.0,1.88,5.58,14.98,21.40,11.0,0.35,2.03,12.59,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,12.0,X-LC,3.32,142822.0,-133701.0,243769.0,-0.51,-48.35,170.68,39.81,54.0,-0.55,1.04,0.00,XY24,NTT,AUTO
51,QUESS,986.00,-48.21,30.0,X-SC,38.41,51410.0,-13596.0,164522.0,0.26,-20.91,320.02,232.18,198.0,-0.08,0.37,0.26,XY24,NTT,MISC
82,VBL,671.64,-21.55,39.0,X-LC,10.68,282762.0,-33080.0,145736.0,0.36,-10.47,51.54,35.67,5.0,-0.23,2.05,1.93,X40N,ATH,FMCG
9,BAJAJHFL,181.50,-15.75,36.0,X-MC,5.66,177464.0,-23042.0,120196.0,0.91,-11.49,67.73,48.45,90.0,-0.19,1.29,2.34,X40N,ATH,FINANCE
73,TCS,4389.97,-30.29,43.0,X-LC,12.35,277817.0,-68139.0,134852.0,0.21,-19.70,48.54,19.28,1.0,-0.51,2.02,2.70,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,30.0,X-SC,38.41,51410.0,-13596.0,164522.0,0.26,-20.91,320.02,232.18,198.0,-0.08,0.37,0.26,XY24,NTT,MISC
55,RELAXO,1176.00,-41.43,34.0,X-SC,3.76,79815.0,-65345.0,141273.0,-0.57,-45.02,177.00,52.31,136.0,-0.46,0.58,6.84,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-37.72,34.0,X-SC,4.42,93168.0,-35502.0,80805.0,-0.15,-27.59,86.73,35.20,219.0,-0.44,0.68,6.94,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-4.94,50.0,X-SC,1.18,108002.0,5998.0,37930.0,2.04,5.88,35.12,43.06,122.0,0.16,0.78,24.00,X40N,ATH,MISC
32,HONAUT,58357.33,-23.12,45.0,X-SC,2.31,108345.0,-19593.0,66730.0,0.07,-15.31,61.59,36.84,143.0,-0.29,0.79,10.92,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-30.29,43.0,X-LC,12.35,277817.0,-68139.0,134852.0,0.21,-19.70,48.54,19.28,1.0,-0.51,2.02,2.70,X40,ATH,IT
39,INFY,2275.00,-19.60,47.0,X-LC,7.65,313579.0,691.0,170995.0,-1.19,0.22,54.53,54.87,3.0,0.00,2.28,6.88,X40,BTT,IT
41,ITC,452.00,-39.61,43.0,X-LC,2.83,195356.0,-4782.0,25670.0,0.68,-2.39,13.14,10.44,4.0,-0.19,1.42,3.04,X40,NTT,FMCG
82,VBL,671.64,-21.55,39.0,X-LC,10.68,282762.0,-33080.0,145736.0,0.36,-10.47,51.54,35.67,5.0,-0.23,2.05,1.93,X40N,ATH,FMCG
1,ABB,7934.00,-40.25,50.0,H-LC,7.41,249696.0,-11923.0,131140.0,1.61,-4.56,52.52,45.57,7.0,-0.09,1.81,6.25,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-45.67,50.0,L-SC,28.75,81600.0,-31949.0,72004.0,0.59,-28.14,88.24,35.27,268.0,-0.44,0.59,99.72,XR,NTT,HOTELS
48,MASFIN,398.61,-21.41,39.0,H-SC,9.86,89565.0,-8415.0,30022.0,-0.35,-8.59,33.52,22.05,152.0,-0.28,0.65,30.19,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,105.37,57.0,H-SC,11.49,129627.0,-12852.0,32368.0,0.90,-9.02,24.97,13.70,163.0,-0.40,0.94,53.00,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,54.0,H-SC,9.93,144891.0,-34180.0,99902.0,-0.36,-19.09,68.95,36.71,135.0,-0.34,1.05,43.76,XR,NTT,BANKS
33,ICICIGI,2260.25,-14.31,65.0,X-MC,1.68,147556.0,11423.0,19699.0,7.50,8.39,13.35,22.86,91.0,0.58,1.07,22.06,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,68.0,M-SC,17.32,148338.0,-882.0,145060.0,0.03,-0.59,97.79,96.62,208.0,-0.01,1.08,60.53,XY25,NTT,FINANCE
68,SURYODAY,240.00,57.89,54.0,H-SC,9.93,144891.0,-34180.0,99902.0,-0.36,-19.09,68.95,36.71,135.0,-0.34,1.05,43.76,XR,NTT,BANKS
31,HINDZINC,730.22,36.19,68.0,M-LC,8.88,221763.0,16687.0,95890.0,0.82,8.14,43.24,54.89,52.0,0.17,1.61,34.19,X5K,ATH,METALS
4,ANGELONE,3033.00,11.73,63.0,X-SC,9.95,197587.0,6581.0,51116.0,0.21,3.45,25.87,30.21,157.0,0.13,1.43,25.05,X40N,NTT,FINANCE
63,SHALBY,327.00,1163.16,53.0,M-SC,22.49,171899.0,-9244.0,54715.0,0.32,-5.10,31.83,25.10,235.0,-0.17,1.25,37.76,XY24,NTT,HEALTHCARE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.50
1,25,44.01
2,50,75.88


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.35
LC    33.73
MC    21.89
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.67
X40      14.82
XY25     12.14
XR       10.95
X40N     10.11
AR        8.23
OX40N     7.69
X200      1.82
X5K       1.61
SR        1.11
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.56
X-LC    22.57
X-MC    16.13
M-SC    12.72
M-LC     5.37
H-LC     4.76
X-SC     4.63
H-MC     3.82
M-MC     1.63
L-SC     1.44
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.07,-5.82,40.70
IT,13.01,-17.28,80.71
FINANCE,12.21,-12.43,63.40
MISC,6.94,-20.15,86.73
BANKS,6.44,-12.55,71.75
PAINTS,5.59,-19.64,37.47
ELECTRICAL,5.38,-11.09,50.06
INSURANCE,3.88,-3.57,42.28
AUTO,2.84,-47.29,117.71


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3350984.0,22
XR,1401403.0,14
AR,1195288.0,9
X40,797593.0,10
X40N,633396.0,8
OX40N,585922.0,10
XY25,508671.0,8
SR,267293.0,2
X5K,95890.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3479539.0,25
M-SC,1289519.0,15
X-LC,1129168.0,13
X-MC,1102991.0,11
X-SC,542376.0,6
H-MC,353445.0,3
H-LC,283936.0,3
M-LC,268870.0,4
L-SC,262108.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1213048.0      6
           XR         830142.0      7
           AR         822885.0      5
M-SC       XY24       697719.0      6
X-MC       XY24       589352.0      4
X-LC       X40        498820.0      5
           XY24       308273.0      2
X-SC       X40N       297049.0      4
H-SC       OX40N      274416.0      4
           SR         267293.0      2
X-MC       X40        217968.0      4
X-LC       X40N       216151.0      3
H-LC       AR         209217.0      2
H-MC       XY24       181949.0      1
X-MC       XY25       175475.0      2
L-SC       XR         172071.0      2
M-MC       XY24       168626.0      1
X-SC       XY24       164522.0      1
M-SC       AR         163186.0      2
           OX40N      149207.0      4
           XY25       145060.0      1
           XR         134347.0      2
X-MC       X40N       120196.0      1
X-LC       XY25       105924.0      3
M-LC       XR         105474.0      1
H-MC       XR          99234.0      1
M-LC       X5K         95890.0      1
L-SC       OX40N       90037.0      1
X-SC       X40         80805.0      1
H-LC       X200        74719.0      1
H-MC       OX40N       72262.0      1
H-SC       MH          71755.0      1
L-MC       XR          60135.0      1
L-LC       XY25        42201.0      1
M-LC       XY25        40011.0      1
           XY24        27495.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
